In [3]:
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
from slsim.Lenses.lens_pop import LensPop
from slsim.Plots.lens_plots import LensingPlots
import numpy as np
import corner
import slsim.Pipelines as pipelines
import slsim.Sources as sources
import slsim.Deflectors as deflectors
from astropy.table import Table, vstack
from tqdm import tqdm
import pickle, gzip
import multiprocessing
import pandas as pd
from warnings import filterwarnings
filterwarnings("ignore", category=UserWarning, append=True)
filterwarnings("ignore", category=RuntimeWarning, append=True)

In [4]:
PROJECT_DIR = "~/projects/PDSPL"

## Generate SLSim Galaxy Galaxy Lenses

### Define source and deflector populations

In [3]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# load red and blue galaxy catalogs
red_galaxy_catalog = Table.read(f"{PROJECT_DIR}/data/skypy_galaxy_catalogs/SKYPY_RED_GALAXIES_{1000.0}_SQDEG_0.fits",
                                  format="fits")
blue_galaxy_catalog = Table.read(f"{PROJECT_DIR}/data/skypy_galaxy_catalogs/SKYPY_BLUE_GALAXIES_{100.0}_SQDEG_0.fits",
                                   format="fits") 

# Initiate deflector population class.
lens_galaxies = deflectors.EllipticalLensGalaxies(
    galaxy_list=red_galaxy_catalog,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light=None,
    cosmo=cosmo,
    sky_area = Quantity(1000.0, "deg2"),
    gamma_pl = {"mean": 2.078, "std_dev": 0.16,}, # Auger et al. 2010
)

# Initiate source population class.
source_galaxies = sources.Galaxies(
    galaxy_list=blue_galaxy_catalog,
    kwargs_cut=kwargs_source_cut,
    cosmo=cosmo,
    sky_area=Quantity(100.0, "deg2"),
    catalog_type="skypy",
    extended_source_type="single_sersic",
)

In [4]:
print("# of red galaxies:", len(red_galaxy_catalog))
print("# of blue galaxies:", len(blue_galaxy_catalog))

# of red galaxies: 84567650
# of blue galaxies: 131301420


### Draw and Save Lenses for 20000 deg2 of sky area

In [11]:
# define a sky area to render lenses for
sky_area = Quantity(value=200, unit="deg2")
scale_factor = 100 # scale factor to get 20000 deg2 worth of lenses

In [ ]:
# make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_population=lens_galaxies,
    source_population=source_galaxies,
    cosmo=cosmo,
    sky_area=sky_area,
)

# we draw from the same population multiple times!
for i in tqdm(range(scale_factor), desc="Drawing lenses"):
    selected_lenses = gg_lens_pop.draw_population(kwargs_lens_cuts={}) # we will apply lens cuts later, so we can save all the lenses and apply different cuts for different analyses
    with gzip.open(f"../data/SLSimLensesNoCuts/SLSIM_GGL_LENSES_{sky_area.value}_SQDEG_{i}.pkl.gz", "wb") as f:
        pickle.dump(selected_lenses, f)

Drawing lenses: 100%|██████████| 100/100 [12:36:23<00:00, 453.84s/it] 


In [ ]:
%%writefile ~/repos/self/pdspl-analysis/notebooks/worker.py
# This entire cell will be saved as 'worker.py'

import numpy as np
import astropy.units as u
import astropy.constants as const
from astropy.cosmology import FlatLambdaCDM
from lenstronomy.LensModel.lens_model import LensModel
import warnings

# Suppress warnings for cleaner logs
warnings.filterwarnings("ignore")

cosmo_true = FlatLambdaCDM(H0=70, Om0=0.3)

def extract_lens_properties(args):
    """
    Worker function that extracts parameters from a lens object and returns them as a dictionary.
    This function is designed to be used in parallel processing.
    """
    lens_id, lens = args
    
    # --- Objects ---
    deflector = lens.deflector
    source = lens.source(index=0)

    # --- System Properties ---
    z_D = lens.deflector_redshift
    z_S = lens.source_redshift_list[0]
    theta_E = lens.einstein_radius[0]
    num_images = lens.image_number[0]

    # --- Source Properties (Geometry) ---
    # needed for Collett 2015 like cuts
    xs, ys = lens.source(0).extended_source_position
    radial_dist_S = np.sqrt(xs**2 + ys**2)
    size_S = source.angular_size 

    # --- Photometry (Source & Deflector) ---
    bands = ['g', 'r', 'i', 'z', 'y']
    mags = {}
    
    for b in bands:
        # 1. Unlensed Source
        mags[f'mag_S_{b}'] = source.extended_source_magnitude(b)
        
        # 2. Lensed Source
        mags[f'mag_S_{b}_lensed'] = lens.extended_source_magnitude(band=b, lensed=True)[0]
        
        # 3. Deflector
        mags[f'mag_D_{b}'] = deflector.magnitude(b)

    # --- Magnification ---
    es_magnification = lens.extended_source_magnification[0]

    # --- Deflector Mass & Geometry ---
    sigma_v_D = deflector.velocity_dispersion()
    stellar_mass_D = deflector.stellar_mass
    e1_mass_D, e2_mass_D = deflector.mass_ellipticity
    e_mass_D = np.sqrt(e1_mass_D**2 + e2_mass_D**2)
    gamma_pl = deflector.halo_properties.get('gamma_pl', 2.0)
    
    # Light Size
    size_D = deflector.angular_size_light # Half-light radius in arcsec

    # --- Advanced Physics (Kappa & Surface Brightness) ---
    lenstronomy_kwargs = lens.lenstronomy_kwargs()
    lens_model_lenstronomy = LensModel(lens_model_list=lenstronomy_kwargs[0]["lens_model_list"])
    lenstronomy_kwargs_lens = lenstronomy_kwargs[1]["kwargs_lens"]
    
    deflector_center = deflector.deflector_center
    grid = np.linspace(-size_D, size_D, 500)
    grid_x, grid_y = np.meshgrid(grid + deflector_center[0], grid + deflector_center[1])
    
    # Kappa Calculation
    kappa_map = lens_model_lenstronomy.kappa(grid_x, grid_y, kwargs=lenstronomy_kwargs_lens)
    mask = np.sqrt((grid_x - deflector_center[0])**2 + (grid_y - deflector_center[1])**2) < size_D / 2
    kappa_within_half_light_radii = np.nanmean(kappa_map[mask])

    D_s = cosmo_true.angular_diameter_distance(z_S)
    D_d = cosmo_true.angular_diameter_distance(z_D)
    D_ds = cosmo_true.angular_diameter_distance_z1z2(z_D, z_S)
    
    sigma_crit = (const.c**2 / (4 * np.pi * const.G)) * (D_s / (D_d * D_ds))
    sigma_crit = sigma_crit.to(u.Msun / u.pc**2).value
    surface_density = sigma_crit * kappa_within_half_light_radii

    # Surface Brightness Calculation (g-band)
    surface_brightness_map = deflector.surface_brightness(grid_x, grid_y, band="g")
    mask_sb = np.sqrt((grid_x - deflector_center[0])**2 + (grid_y - deflector_center[1])**2) < size_D
    mean_surface_brightness = np.nanmean(surface_brightness_map[mask_sb])

    # Physical Radius
    R_e_kpc_val = (cosmo_true.kpc_proper_per_arcmin(z_D) * \
                    ((size_D * u.arcsec).to(u.arcmin))).to(u.kpc).value

    # --- Contrast Ratios ---
    contrasts = {}
    for b in bands:
        cr_raw = lens.contrast_ratio(band=b, source_index=0)
        cr_padded = np.array(list(cr_raw) + [np.nan] * (4 - len(cr_raw)))
        contrasts[f'contrast_ratio_{b}'] = cr_padded

    return {
        "lens_id": lens_id, 
        "z_D": z_D, 
        "z_S": z_S, 
        "theta_E": theta_E, 
        "num_images": num_images,
        
        # Source (Union of geometry + photometry)
        "radial_dist_S": radial_dist_S,    # = sqrt(x_S^2 + y_S^2)
        "size_S": size_S,                  # Angular Size of Source (arcsec)
        **mags,                            # Contains mag_S_i AND mag_S_i_lensed AND mag_D_i for i in g,r,i,z,y bands
        "es_magnification": es_magnification,

        # Deflector Light
        "R_e_arcsec": size_D,                # Effective radius in arcsec
        "surf_bri_mag/arcsec2": mean_surface_brightness,

        # Deflector Mass
        "sigma_v_D": sigma_v_D,
        "stellar_mass_D": stellar_mass_D,
        "e1_mass_D": e1_mass_D, 
        "e2_mass_D": e2_mass_D, 
        "e_mass_D": e_mass_D,
        "gamma_pl": gamma_pl, 
        
        # Physics
        "R_e_kpc": R_e_kpc_val, 
        "Sigma_half_Msun/pc2": surface_density,
        
        # Observables
        **contrasts
    }

Writing /home/paras/repos/self/pdspl-analysis/notebooks/worker.py


In [8]:
# --- Import the worker function from the file we just created ---
# from projects.PDSPL.notebooks.worker import extract_lens_properties
from worker import extract_lens_properties

In [ ]:
main_progress_bar = tqdm(
    range(scale_factor), desc="Overall Progress", position=0)

for i in main_progress_bar:
    # 1. LOAD LENSES
    with gzip.open(f"../data/SLSimLensesNoCuts/SLSIM_GGL_LENSES_{sky_area.value}_SQDEG_{i}.pkl.gz", "rb") as f:
        loaded_lenses = pickle.load(f)

    # 2. RUN IN PARALLEL
    num_processes = multiprocessing.cpu_count()-3
    multiprocessing.set_start_method("spawn", force=True)

    with multiprocessing.Pool(processes=num_processes) as pool:
        lens_ids = [f"lens_{i}_{j}" for j in range(len(loaded_lenses))]
        lens_args = list(zip(lens_ids, loaded_lenses))
        results_list = list(tqdm(
            pool.imap(extract_lens_properties, lens_args),
            total=len(loaded_lenses),
            desc="Calculating lens properties"
        ))

    # 3. PROCESS RESULTS
    results_list = [res for res in results_list if res is not None]

    if results_list:
        df = pd.DataFrame(results_list)
        final_table = Table.from_pandas(df)
        
        # Save the table to a FITS file
        output_path = f"../data/SLSimLensesNoCutsCatalogs/GGL_{sky_area.value}_SQDEG_{i}.fits"
        final_table.write(output_path, format="fits", overwrite=True)

    else:
        print("       No results were generated.")

Overall Progress:  99%|█████████▉| 99/100 [25:33:50<15:31, 931.84s/it]

## Extract Catalog of Lenses for Different Surveys

In [5]:
def satisfies_Collett_2015_cuts_table(lens_catalog_table, 
                                      seeing = 0.5, 
                                      snr_threshold = {'i': 20},
                                      redshift_limit_source = None,
                                      contrast_ratio_threshold_i_band = None,
                                      return_boolean_array = False):
    """
    Checks whether lenses in a given lens catalog table satisfy the Collett 2015 (https://iopscience.iop.org/article/10.1088/0004-637X/811/1/20) cuts.

    Parameters:
    lens_catalog_table: an Astropy Table containing lens properties
    seeing: the seeing in arcseconds (default 0.5 for LSST Optimal Seeing)
    snr_threshold: a dictionary of SNR thresholds for each band (default {'i': 20} for LSST)
    redshift_limit_source: optional redshift limit for the source (default None, meaning no cut)
    contrast_ratio_threshold_i_band: optional contrast ratio threshold for the i-band (default None, meaning no cut)

    Returns:
    A table of lenses that satisfy the cuts, or if return_boolean_array is True, a boolean array indicating which lenses satisfy the cuts.
    """
    conditions = np.ones(len(lens_catalog_table), dtype=bool)

    # Criteria 1: Multiple imaging
    condition_1 = lens_catalog_table['radial_dist_S'] < lens_catalog_table['theta_E']
    conditions &= condition_1

    # Criteria 2: Image Resolution
    condition_2 = lens_catalog_table['theta_E'] > np.sqrt(lens_catalog_table['size_S']**2 + (seeing/2)**2)
    conditions &= condition_2

    # Criteria 3: Tangential Arc Resolution
    condition_3 = lens_catalog_table['es_magnification'] * lens_catalog_table['size_S'] > seeing
    conditions &= condition_3

    # Criteria 4: Magnification
    condition_4 = lens_catalog_table['es_magnification'] > 3
    conditions &= condition_4

    # Criteria 5: SNR > 20
    if snr_threshold is not None:
        condition_5 = np.ones(len(lens_catalog_table), dtype=bool)
        for band, threshold in snr_threshold.items():
            snr_band = lens_catalog_table[f'snr_{band}'].copy()
            snr_band[snr_band == None] = 0  # Treat None SNR values as 0 for the purpose of this cut
            condition_band = snr_band > threshold
            condition_5 &= condition_band
        conditions &= condition_5
    
    # Criteria 6: Contrast Ratio (not in original Collett 2015 cuts, but often used in lens selection)
    # at least two images with contrast ratio > threshold in the i-band
    if contrast_ratio_threshold_i_band is not None:
        conditions_6 = np.ones(len(lens_catalog_table), dtype=bool) # initialize to True
        contrast_ratio_i = lens_catalog_table['contrast_ratio_i'] # this is in mags I_source_light/I_lens_light [mag/arcsec^2]
        for i in range(len(lens_catalog_table)):
            contrast_ratio_i_lens = contrast_ratio_i[i]
            
            contrast_ratio_i_lens = 10**(-contrast_ratio_i_lens / 2.5) # convert to flux ratio # this is an array for each image
            contrast_ratio_i_lens = contrast_ratio_i_lens[~np.isnan(contrast_ratio_i_lens)] # remove nan values (for images that don't exist)
            num_images_contrast_ok = np.sum(contrast_ratio_i_lens > contrast_ratio_threshold_i_band)
            if num_images_contrast_ok < 2:
                conditions_6[i] = False
        conditions &= conditions_6

    # Criteria 7: Redshift cuts for Source if needed (e.g. for 4MOST)
    if redshift_limit_source is not None:
        condition_6 = lens_catalog_table['z_S'] < redshift_limit_source
        conditions &= condition_6
    
    if return_boolean_array:
        return conditions
    
    return lens_catalog_table[conditions]

### LSST Y10 - SNR and contrast ratio cuts included with seeing = 0.5"

In [13]:
sky_area = Quantity(200, "deg2")
scale_factor = 100 # scale factor to get 20000 deg2 worth of lenses
# load saved catalogs and apply cuts to generate final GGL catalogs for analysis

GGL_catalog_no_SNR_cut = None # we will update this with the first catalog that we load

for i in tqdm(range(scale_factor), desc="Applying Collett 2015 cuts to catalogs"):
    # 1. LOAD CATALOG
    lenses_path = f"../data/SLSimLensesNoCuts/SLSIM_GGL_LENSES_{sky_area.value}_SQDEG_{i}.pkl.gz"
    catalog_path = f"../data/SLSimLensesNoCutsCatalogs/GGL_{sky_area.value}_SQDEG_{i}.fits"
    lens_catalog_table = Table.read(catalog_path, format="fits")
    with gzip.open(lenses_path, "rb") as f:
        loaded_lenses = pickle.load(f)

    # 2. APPLY CUTS
    filtered_table = satisfies_Collett_2015_cuts_table(lens_catalog_table, 
                                                       seeing = 0.5, snr_threshold = None, 
                                                       redshift_limit_source = None,
                                                       contrast_ratio_threshold_i_band = 2)

    # 3. Add SNR columns to the filtered table
    snr_dict = {}
    for band in ['g', 'r', 'i', 'z', 'y']:
        snr_dict[band] = np.zeros(len(filtered_table))
        # filtered_table[f'snr_{band}'] = np.zeros(len(filtered_table))
    
    for lens_id in filtered_table['lens_id']: 
        idxs = lens_id.split("_") # lens_id format is "lens_{i}_{j}" where i is the catalog number and j is the lens number within that catalog
        lens_j = int(idxs[2])
        lens = loaded_lenses[lens_j]
        theta_E = lens.einstein_radius[0]
        for band in ['g', 'r', 'i', 'z', 'y']:
            snr_band = lens.snr(
                band=band,
                fov_arcsec=np.max([theta_E * 4, 1]),
                observatory='LSST',
                snr_per_pixel_threshold=1,
            )
            snr_dict[band][filtered_table['lens_id'] == lens_id] = snr_band
    
    # Add SNR columns to the filtered table
    for band in ['g', 'r', 'i', 'z', 'y']:
        filtered_table[f'snr_{band}'] = snr_dict[band]


    if GGL_catalog_no_SNR_cut is None:
        GGL_catalog_no_SNR_cut = filtered_table
    else:
        GGL_catalog_no_SNR_cut = vstack([GGL_catalog_no_SNR_cut, filtered_table])

# filter on SNR to get final catalog for analysis LSST Y10
GGL_catalog_final = GGL_catalog_no_SNR_cut[
    (GGL_catalog_no_SNR_cut['snr_g'] > 20) &
    (GGL_catalog_no_SNR_cut['snr_r'] > 20) &
    (GGL_catalog_no_SNR_cut['snr_i'] > 20)
    ]
GGL_catalog_final.write("../data/GGL_Catalogs/GGL_20000.0_SQDEG_LSSTY10_SNR_20_seeing_0.5_contrast_02.fits", format="fits", overwrite=True)

Applying Collett 2015 cuts to catalogs: 100%|██████████| 100/100 [34:13<00:00, 20.53s/it]


### LSST Y10 - No SNR and No contrast ratio cuts, with seeing = 0.6"

In [16]:
sky_area = Quantity(200, "deg2")
scale_factor = 100 # scale factor to get 20000 deg2 worth of lenses
# load saved catalogs and apply cuts to generate final GGL catalogs for analysis

GGL_catalog_no_SNR_cut = None # we will update this with the first catalog that we load

for i in tqdm(range(scale_factor), desc="Applying Collett 2015 cuts to catalogs"):
    # 1. LOAD CATALOG
    lenses_path = f"../data/SLSimLensesNoCuts/SLSIM_GGL_LENSES_{sky_area.value}_SQDEG_{i}.pkl.gz"
    catalog_path = f"../data/SLSimLensesNoCutsCatalogs/GGL_{sky_area.value}_SQDEG_{i}.fits"
    lens_catalog_table = Table.read(catalog_path, format="fits")
    with gzip.open(lenses_path, "rb") as f:
        loaded_lenses = pickle.load(f)

    # 2. APPLY CUTS
    filtered_table = satisfies_Collett_2015_cuts_table(lens_catalog_table, seeing = 0.6, snr_threshold = None, redshift_limit_source = None)

    # 3. Add SNR columns to the filtered table
    snr_dict = {}
    for band in ['g', 'r', 'i', 'z', 'y']:
        snr_dict[band] = np.zeros(len(filtered_table))
        # filtered_table[f'snr_{band}'] = np.zeros(len(filtered_table))
    
    for lens_id in filtered_table['lens_id']: 
        idxs = lens_id.split("_") # lens_id format is "lens_{i}_{j}" where i is the catalog number and j is the lens number within that catalog
        lens_j = int(idxs[2])
        lens = loaded_lenses[lens_j]
        theta_E = lens.einstein_radius[0]
        for band in ['g', 'r', 'i', 'z', 'y']:
            snr_band = lens.snr(
                band=band,
                fov_arcsec=np.max([theta_E * 4, 1]),
                observatory='LSST',
                snr_per_pixel_threshold=1,
            )
            snr_dict[band][filtered_table['lens_id'] == lens_id] = snr_band
    
    # Add SNR columns to the filtered table
    for band in ['g', 'r', 'i', 'z', 'y']:
        filtered_table[f'snr_{band}'] = snr_dict[band]

    if GGL_catalog_no_SNR_cut is None:
        GGL_catalog_no_SNR_cut = filtered_table
    else:
        GGL_catalog_no_SNR_cut = vstack([GGL_catalog_no_SNR_cut, filtered_table])

GGL_catalog_no_SNR_cut.write("../data/GGL_Catalogs/GGL_20000.0_SQDEG_LSSTY10_SNR_no_seeing_0.6_contrast_no.fits", format="fits", overwrite=True)

Applying Collett 2015 cuts to catalogs: 100%|██████████| 100/100 [34:32<00:00, 20.73s/it]


### 4MOST - No contrast ratio cuts, with seeing = 0.2", SNR > 20

We choose seeing = 0.2" as the lenses will be first discovered in space or ground based imaging surveys (e.g. Euclid, Roman, LSST) and then followed up with 4MOST for spectroscopy. The 0.2" seeing cut is a union of the 0.5" seeing cut for LSST and the 0.18" seeing cut for space based surveys, to ensure that we are including all lenses that could be discovered in imaging surveys and then followed up with 4MOST.

After these cuts we choose the brightest 10000 lenses for the 4MOST catalog, as 4MOST will likely only be able to follow up a subset of the lenses discovered in imaging surveys. And the brightest half from those will have velocity dispersion measurements.

In [6]:
def satisfies_4MOST_cuts_table(lens_catalog_table):
    """
    Returns a table of lenses that satisfy the 4MOST cuts.
    Uses 0.2" seeing, no SNR cut, no contrast ratio cut, 
    and a redshift cut of z_S < 1.5 for the source (to ensure OII is no redshifted).
    """
    filtered_catalog = lens_catalog_table[
        (lens_catalog_table['mag_S_r_lensed'] < 24) &
        (lens_catalog_table['mag_D_r'] < 20) &
        (lens_catalog_table['z_S'] < 1.5)
    ]
    return satisfies_Collett_2015_cuts_table(filtered_catalog, seeing = 0.2, snr_threshold = None, contrast_ratio_threshold_i_band = None)

In [7]:
sky_area = Quantity(200, "deg2")
scale_factor = 100 # scale factor to get 20000 deg2 worth of lenses
# load saved catalogs and apply cuts to generate final GGL catalogs for analysis

GGL_catalog_no_SNR_cut = None # we will update this with the first catalog that we load

for i in tqdm(range(scale_factor), desc="Applying Collett 2015 cuts to catalogs"):
    # 1. LOAD CATALOG
    lenses_path = f"../data/SLSimLensesNoCuts/SLSIM_GGL_LENSES_{sky_area.value}_SQDEG_{i}.pkl.gz"
    catalog_path = f"../data/SLSimLensesNoCutsCatalogs/GGL_{sky_area.value}_SQDEG_{i}.fits"
    lens_catalog_table = Table.read(catalog_path, format="fits")
    with gzip.open(lenses_path, "rb") as f:
        loaded_lenses = pickle.load(f)

    # 2. APPLY CUTS
    filtered_table = satisfies_4MOST_cuts_table(lens_catalog_table)

    # 3. Add SNR columns to the filtered table
    snr_dict = {}
    for band in ['g', 'r', 'i', 'z', 'y']:
        snr_dict[band] = np.zeros(len(filtered_table))
        # filtered_table[f'snr_{band}'] = np.zeros(len(filtered_table))
    
    for lens_id in filtered_table['lens_id']: 
        idxs = lens_id.split("_") # lens_id format is "lens_{i}_{j}" where i is the catalog number and j is the lens number within that catalog
        lens_j = int(idxs[2])
        lens = loaded_lenses[lens_j]
        theta_E = lens.einstein_radius[0]
        for band in ['g', 'r', 'i', 'z', 'y']:
            snr_band = lens.snr(
                band=band,
                fov_arcsec=np.max([theta_E * 4, 1]),
                observatory='LSST',
                snr_per_pixel_threshold=1,
            )
            snr_dict[band][filtered_table['lens_id'] == lens_id] = snr_band
    
    # Add SNR columns to the filtered table
    for band in ['g', 'r', 'i', 'z', 'y']:
        filtered_table[f'snr_{band}'] = snr_dict[band]


    if GGL_catalog_no_SNR_cut is None:
        GGL_catalog_no_SNR_cut = filtered_table
    else:
        GGL_catalog_no_SNR_cut = vstack([GGL_catalog_no_SNR_cut, filtered_table])

# filter on SNR to get final catalog for analysis 4MOST
GGL_catalog_final = GGL_catalog_no_SNR_cut[
    (GGL_catalog_no_SNR_cut['snr_g'] > 20) &
    (GGL_catalog_no_SNR_cut['snr_r'] > 20) &
    (GGL_catalog_no_SNR_cut['snr_i'] > 20)
    ]
GGL_catalog_final.write("../data/GGL_Catalogs/GGL_20000.0_SQDEG_4MOST_SNR_20_seeing_0.2_contrast_no.fits", format="fits", overwrite=True)

Applying Collett 2015 cuts to catalogs: 100%|██████████| 100/100 [14:06<00:00,  8.46s/it]
